In [ ]:
import sys
import logging
import math
import random
from collections import defaultdict
from itertools import combinations
!pip install gurobipy
import gurobipy as gp
from gurobipy import GRB



In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import torch
from gurobipy import Model, GRB, quicksum

In [ ]:

# 첫 번째 문제만 시각화
depot_xy = ([[0.5000, 0.5000]])  # (1, 1, 2)에서 (1, 2)로 축소
node_xy = ([[[0.1663, 0.9714],
         [0.8510, 0.0303],
         [0.2379, 0.1516],
         [0.8557, 0.1405],
         [0.9175, 0.1394],
         [0.9071, 0.2275],
         [0.6006, 0.5054],
         [0.9538, 0.6123],
         [0.4682, 0.9465],
         [0.7295, 0.9662]]])  # (1, problem_size, 2)에서 (problem_size, 2)로 축소
node_demand = ([[0.1326, 0.2337, 0.1557, 0.1448, 0.0193, 0.0984, 0.0987, 0.1961, 0.2474,
         0.2150]])  # (1, problem_size)에서 (problem_size,)로 축소


# 데이터 변환
depot_xy = np.array(depot_xy)  # list를 numpy 배열로 변환
node_xy = np.array(node_xy).squeeze(0)  # (1, problem_size, 2) → (problem_size, 2)
node_demand = np.array(node_demand).squeeze(0)  # (1, problem_size) → (problem_size,)

problem_size = len(node_demand)  # 노드 개수

# 시각화
plt.figure(figsize=(8, 5))

# 창고 (depot) 위치 표시
plt.plot(depot_xy[0, 0], depot_xy[0, 1], 'rs', markersize=10, label='Depot')  # 창고는 빨간 사각형으로 표시

# 고객 노드 위치 표시
plt.scatter(node_xy[:, 0], node_xy[:, 1], c='b', label='Customers')  # 고객은 파란 점으로 표시

# 각 고객의 수요를 텍스트로 표시
for i in range(problem_size):
    plt.text(node_xy[i, 0], node_xy[i, 1], f'{node_demand[i]:.2f}', fontsize=9, ha='right')

# 그래프 설정
plt.xlabel("X Coordinate")
plt.ylabel("Y Coordinate")
plt.title("Customer Locations with Demand")
plt.legend()
plt.grid(True)
plt.show()


In [ ]:
# 거리 계산 (distances 딕셔너리)
distances = {}
all_nodes = np.vstack((depot_xy, node_xy))  # 창고와 고객 노드를 모두 포함한 좌표 배열
n = problem_size  # 고객 노드 수
for i in range(n+1):
    for j in range(n+1):
        distances[(i, j)] = np.hypot(all_nodes[i, 0] - all_nodes[j, 0], all_nodes[i, 1] - all_nodes[j, 1])

# 파라미터 설정
C = list(range(1, n+1))  # 고객 노드 집합 (1부터 n까지)
D = 0  # 창고 노드
d = {i: node_demand[i-1] for i in C}  # 고객 수요 (depot 수요는 없음)
MaxP = 1  # 드론의 최대 적재 용량
MaxSOC = 100  # 배터리 최대 용량
RC = 15  # 창고로 돌아올 때의 최소 배터리 잔량
alpha = 2.29705  # 거리당 배터리 소모량 상수
beta = 3.87886  # 추가 소모 상수
max_distance = max(distances[i, j] for i in range(n+1) for j in range(n+1))  # 노드 간 최대 거리
M = 100 - RC + (alpha * MaxP + beta) * max_distance
print(all_nodes)
print(distances)

In [ ]:
model = gp.Model("DroneOptimization")
# 5. 변수 정의
x = model.addVars(n+1, n+1, vtype=GRB.BINARY, name="x")  # 경로 선택 변수
l = model.addVars(n+1, lb=0, ub=MaxP+0.00000001, vtype=GRB.CONTINUOUS, name="load")  # 적재량
soc = model.addVars(n+1, lb=0, vtype=GRB.CONTINUOUS, name="soc")  # 배터리 잔량
y = model.addVars(C, vtype=GRB.CONTINUOUS, name="y")  # sub-tour 방지 변수
print(x)

In [ ]:
# 6. 목적 함수 (1): 이동 거리 최소화
model.setObjective(gp.quicksum(distances[i, j] * x[i, j] for i in range(n+1) for j in range(n+1)), GRB.MINIMIZE)
#모든 경로의 쌍을 나열한다음 하단 제약조건들을 만족하는 것들만 추출하여 경로 최소화
# 제약 조건 (2) 각 고객은 반드시 한 번만 방문
for i in C:
    model.addConstr(gp.quicksum(x[i, j] for j in range(n+1) if j != i) == 1, name="1")
    #i ==1 일때 j가 1부터 20까지 중 단 1개의 값만이 유효하여 1이라는 값이 나올 수 있음
    # 제약 조건 (3) 각 고객은 한 번만 방문되도록
for j in C:
    model.addConstr(gp.quicksum(x[i, j] for i in range(n+1) if i != j) == 1, name="2")
    #j ==1 일때 i가 1부터 20까지 중 단 1개의 값만이 유효하여 1이라는 값이 나올 수 있음
    # 제약 조건 (4) 드론이 고객에 도착하면 반드시 떠나야 함
for i in C:
    model.addConstr(gp.quicksum(x[i, j] for j in range(n+1) if j != i) == gp.quicksum(x[j, i] for j in range(n+1) if j != i), name="3")
    print(type(x))
print(x)
model.addConstr(gp.quicksum(x[D, i] for i in C) == gp.quicksum(x[i, D] for i in C),name="4")
#(5) j(D)로 들어오는 노드와 j(D) 에서 나가는 드론의 합은 같음

In [ ]:
# (6) Depot에서 출발하는 드론은 무조건 maxP를 가지고 나감
for j in C:
    model.addConstr((x[D, j] == 1) >> (l[D] == MaxP), name=f"depot_capacity_refill_{j}")
    # (7) 드론의 load는 고객의 demand에 맞게 감소하며, load는 항상 demand보다 커야함
V= range(n+1)
model.addConstrs((x[i, j] == 1) >> (l[i] - d[j] == l[j]) for i in V for j in C if i != j )
model.addConstrs(l[i] >= d[i] for i in C)

In [ ]:
model.addConstr(soc[D] == MaxSOC, name="SOC_initial")

M = 1000  # 충분히 큰 상수 설정

model.addConstrs(
    soc[j] <= soc[i] - distances[i, j] * (alpha * l[i] + beta) + M * (1 - x[i, j])
    for i in range(n+1) for j in  C if i != j
)

model.addConstrs(
    soc[j] >= soc[i] - distances[i, j] * (alpha * l[i] + beta) - M * (1 - x[i, j])
    for i in range(n+1) for j in range(n+1) if i != j
)

# 제약 조건 (10) 창고로 돌아올 때 최소한의 배터리 용량 확보
for i in C:
    model.addConstr(RC <= soc[i] - distances[i, D] * (alpha* l[i] + beta) + M * (1 - x[i, D]), name="9")



In [ ]:
# 제약 조건 (11) sub-tour 방지
for i in C:
    for j in C:
        if i != j:
            model.addConstr(y[i] - y[j] + (n+1) * x[i, j] <= n, name="10")
# 제약 조건 (12) 자기 자신으로의 경로는 불가능
for i in range(n+1):
    model.addConstr(x[i, i] == 0, name="11")
# 제약 조건 (13) 변수 값 범위 설정
for i in range(n+1):
    for j in range(n+1):
        model.addConstr(x[i, j] >= 0, name="13")
        model.addConstr(x[i, j] <= 1, name="14")
    model.addConstr(l[i] >= 0, name="15")
    model.addConstr(soc[i] >= 0, name="16")
    model.addConstr(soc[i] <= MaxSOC, name="17")

In [ ]:
# 8. 모델 최적화 실행
model.optimize()

In [ ]:
for i in range(n+1):
    for j in range(n+1):
        if x[i, j].x > 0.5:  # 값이 0.5 이상이면 경로가 선택된 것으로 간주
            print(f"Path selected: {i} -> {j}")

In [ ]:
# 모델 최적화가 성공했는지 확인
if model.status == GRB.OPTIMAL:
    # 최적 경로를 저장할 리스트
    selected_edges = []

    # 최적 경로 변수 추출
    for i in range(problem_size + 1):  # 고객 노드 + 창고 노드
        for j in range(problem_size + 1):
            # x[i, j] 변수가 최적 경로에 포함된 경우 (즉, 값이 1인 경우)
            var = model.getVarByName(f"x[{i},{j}]")
            if var.x > 0.5:
                selected_edges.append((i, j))

    # 시각화
    plt.figure(figsize=(8, 8))

    # 창고와 고객 위치 표시
    plt.plot(depot_xy[0, 0], depot_xy[0, 1], 'rs', markersize=10, label='Depot')  # 창고는 빨간 사각형
    plt.scatter(node_xy[:, 0], node_xy[:, 1], c='b', label='Customers')  # 고객은 파란 점

    # 각 노드 번호 및 수요 텍스트로 표시
    all_nodes = np.vstack([depot_xy, node_xy])  # 창고와 고객 위치를 하나로 합침
    for i in range(problem_size + 1):
        z, y = all_nodes[i]
        if i == 0:
            plt.text(z, y, f'Depot', fontsize=9, color='red', ha='right')
        else:
            plt.text(z, y, f'Node {i}\n({node_demand[i-1]:.2f})', fontsize=8, ha='right', color='blue')

    # 최적 경로 그리기
    for i, j in selected_edges:
        plt.plot([all_nodes[i][0], all_nodes[j][0]], [all_nodes[i][1], all_nodes[j][1]], 'k-', lw=1)

    # 그래프 설정
    plt.xlabel("X Coordinate")
    plt.ylabel("Y Coordinate")
    plt.title("Optimal Route with Node Numbers")
    plt.legend()
    plt.grid(True)
    plt.show()

else:
    print("Optimal solution not found.")